In [1]:
import speech_recognition as sr
import pyttsx3
import webbrowser
import platform
import openai
import os
import datetime
import random
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

class JarvisPersonality:
    GREETINGS = [
        "Ah, there you are! How can I assist you today?",
        "Good to hear your voice! What's on your mind?",
        "At your service! What can I do for you?",
        "Ready and waiting! How may I help?"
    ]
    
    FAREWELLS = [
        "Until next time, remember: The best way to predict the future is to create it!",
        "Signing off - but I'm always here when you need me!",
        "Before I go, here's a thought: {random_thought}",
        "Catch you on the flip side! Don't forget to hydrate!"
    ]
    
    COMPLIMENTS = [
        "That's an excellent question!",
        "You always ask the most interesting things!",
        "I was just thinking about that myself!",
        "Your curiosity is inspiring!"
    ]

class JarvisBrain:
    def __init__(self):
        self.conversation_history = []
        self.user_name = None
        self.last_action = None
        self.personality = JarvisPersonality()
        
    def remember(self, role, content):
        self.conversation_history.append({"role": role, "content": content})
        
    def get_context(self):
        return self.conversation_history[-6:]  # Last 3 exchanges
        
    def generate_response(self, prompt):
        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": self._create_system_prompt()},
                    *self.get_context(),
                    {"role": "user", "content": prompt}
                ],
                temperature=0.9,
                max_tokens=150
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            return f"I seem to be having a brain freeze. Could you repeat that?"

    def _create_system_prompt(self):
        return f"""
        You are Jarvis, a highly advanced AI assistant with a charming personality.
        User's name: {self.user_name or 'Unknown'}
        Current time: {datetime.datetime.now().strftime("%H:%M")}
        Personality traits: witty, helpful, slightly humorous
        Recent context: {self.last_action or 'No recent actions'}
        """

class JarvisVoice:
    def __init__(self):
        self.engine = pyttsx3.init()
        self.engine.setProperty('rate', 180)
        voices = self.engine.getProperty('voices')
        self.engine.setProperty('voice', voices[0].id)  # Male voice
        
    def speak(self, text, emotional_tone='neutral'):
        # Add emotional inflection
        if emotional_tone == 'excited':
            self.engine.setProperty('rate', 220)
        elif emotional_tone == 'calm':
            self.engine.setProperty('rate', 150)
            
        print(f"Jarvis: {text}")
        self.engine.say(text)
        self.engine.runAndWait()
        # Non-blocking runAndWait for Jupyter Notebooks
        if platform.system() == 'Windows':
            import pythoncom
            pythoncom.CoInitialize()
        self.engine.setProperty('rate', 180)  # Reset to normal

def setup_chrome():
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-notifications")
    return webdriver.Chrome(options=options)

def humanized_listen():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("\n🌀 Listening...")
        r.pause_threshold = 1.2
        r.adjust_for_ambient_noise(source, duration=1)
        try:
            audio = r.listen(source, timeout=8)
            print("🔍 Processing...")
            text = r.recognize_google(audio).lower()
            print(f"You: {text}")
            return text
        except sr.WaitTimeoutError:
            return ""
        except Exception as e:
            print(f"Audio error: {str(e)}")
            return ""

def handle_chrome_interaction(driver, query):
    try:
        driver.get("https://www.google.com")
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "q"))
        ).send_keys(query + Keys.RETURN)
        return True
    except Exception as e:
        print(f"Chrome error: {str(e)}")
        return False

def main():
    brain = JarvisBrain()
    voice = JarvisVoice()
    chrome_driver = None
    
    # Initialization sequence
    voice.speak("Initializing my systems...", 'calm')
    voice.speak("Loading my personality matrix...", 'calm')
    voice.speak("""Hello... I am Jarvis! Version-2.0...""", 'calm')
    
    # Get user name with personality
    voice.speak("Before we begin, may I know how to address you?", 'calm')
    while not brain.user_name:
        name_input = humanized_listen()
        if name_input:
            if "my name is" in name_input:
                brain.user_name = name_input.split("my name is")[-1].strip().title()
            else:
                brain.user_name = name_input.strip().title()
            voice.speak(f"A pleasure to meet you, {brain.user_name}! {random.choice(JarvisPersonality.GREETINGS)}", 'excited')
            brain.remember('system', f"User's preferred name: {brain.user_name}")
    
    # Main interaction loop
    while True:
        command = humanized_listen().strip()
        
        if not command:
            continue
            
        if "exit" in command or "bye" in command:
            farewell = random.choice(JarvisPersonality.FAREWELLS)
            if "{random_thought}" in farewell:
                farewell = farewell.format(random_thought=brain.generate_response("Give an inspiring short quote"))
            voice.speak(farewell, 'calm')
            if chrome_driver:
                chrome_driver.quit()
            break
            
        if "open chrome" in command:
            voice.speak("Opening the gateway to knowledge...", 'excited')
            chrome_driver = setup_chrome()
            brain.last_action = "Opened Chrome browser"
            
        elif "search for" in command:
            query = command.replace("search for", "").strip()
            if chrome_driver and handle_chrome_interaction(chrome_driver, query):
                voice.speak(f"Here's what I found about {query}", 'excited')
                brain.last_action = f"Searched for: {query}"
            else:
                voice.speak("My search engines seem to be offline. Let's try that again later?", 'calm')
                
        else:
            # Conversational AI
            voice.speak(random.choice(JarvisPersonality.COMPLIMENTS), 'excited')
            response = brain.generate_response(command)
            brain.remember('assistant', response)
            voice.speak(response)

if __name__ == "__main__":
    main()

Jarvis: Initializing my systems...
Jarvis: Loading my personality matrix...
Jarvis: Hello... I am Jarvis! Version-2.0...
Jarvis: Before we begin, may I know how to address you?

🌀 Listening...
🔍 Processing...
You: santosh
Jarvis: A pleasure to meet you, Santosh! Good to hear your voice! What's on your mind?

🌀 Listening...
🔍 Processing...
You: give me the synopsis of telugu movie kodama simham
Jarvis: That's an excellent question!
Jarvis: "Kodama Simham", a Telugu movie released in 1990, is an action-packed drama featuring the superstar Chiranjeevi in the lead role. It's an engaging story of bravery, justice, and love.

Chiranjeevi plays the character of Bhupati Rayudu, a righteous cowboy who tries to protect his land from the wrong doers. Trouble arises when a corrupt sheriff tries to gain control over Rayudu's land. Rayudu's life takes a dramatic turn when he meets Anu (Radha), a city girl who gradually falls in love with him.

As the story progresses, Rayudu stands against the power